# Play with implicit 1D diffusion: NumPy vs Numba (Thomas solver)

This notebook benchmarks three things:

1. **Baseline**: dense matrices + `np.linalg.solve` (works, but ignores tridiagonal structure)
2. **Structured NumPy**: tridiagonal RHS + Thomas solver (O(N))
3. **Structured Numba**: same Thomas solver JITed

The important shift is: *Numba can only help once the expensive work is in Python loops (i.e., your solver), not inside LAPACK.*


In [1]:
import numpy as np
from numba import njit

# Reproducibility
np.random.seed(0)


In [2]:
# Parameters
N = 51          # grid points including boundaries
L = 1.0
nsteps = 100    # benchmark steps (change freely)
dt = 1e-4

dx = L/(N-1)
r = dt/(dx*dx)

x = np.linspace(0.0, L, N)
u0 = np.sin(np.pi * x)


In [3]:
# Build dense A, B for baseline (interior unknowns only)

def build_dense_mats(N, r):
    n = N - 2
    main_A = (2 + 2*r) * np.ones(n)
    off_A  = (-r) * np.ones(n-1)
    A = np.diag(main_A) + np.diag(off_A, 1) + np.diag(off_A, -1)

    main_B = (2 - 2*r) * np.ones(n)
    off_B  = (r) * np.ones(n-1)
    B = np.diag(main_B) + np.diag(off_B, 1) + np.diag(off_B, -1)
    return A, B

A, B = build_dense_mats(N, r)


In [4]:
# Baseline: dense solve each step

def step_numpy_dense(A, B, u, nsteps):
    u = u.copy()
    for _ in range(nsteps):
        rhs = B @ u[1:-1]
        u[1:-1] = np.linalg.solve(A, rhs)
    return u

# Warm-up
_ = step_numpy_dense(A, B, u0, 2)


In [5]:
%%timeit
_ = step_numpy_dense(A, B, u0, nsteps)


4.56 ms ± 65.2 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
# Tridiagonal representation for A and B
# A has: a=-r (sub), b=2+2r (main), c=-r (super)
# B has: a=+r (sub), b=2-2r (main), c=+r (super)

n = N - 2

aA = (-r) * np.ones(n-1)
bA = (2 + 2*r) * np.ones(n)
cA = (-r) * np.ones(n-1)

aB = ( r) * np.ones(n-1)
bB = (2 - 2*r) * np.ones(n)
cB = ( r) * np.ones(n-1)


def rhs_tridiag(a, b, c, u_int):
    # (a,b,c) are sub/main/super for an n×n tridiagonal
    # u_int is length n
    n = u_int.size
    out = b * u_int
    out[1:]  += a * u_int[:-1]
    out[:-1] += c * u_int[1:]
    return out


In [ ]:
# Thomas solver (NumPy)
# This uses a tridiagonal (Thomas) solver for the implicit 1D diffusion step.
# Dense np.linalg.solve is O(N^3) and was replaced for performance reasons.


def thomas_numpy(a, b, c, d):
    # Solve tridiagonal system with subdiag a (n-1), diag b (n), superdiag c (n-1)
    n = d.size
    cp = np.empty(n-1, dtype=d.dtype)
    dp = np.empty(n, dtype=d.dtype)

    cp[0] = c[0] / b[0]
    dp[0] = d[0] / b[0]

    for i in range(1, n-1):
        denom = b[i] - a[i-1] * cp[i-1]
        cp[i] = c[i] / denom
        dp[i] = (d[i] - a[i-1] * dp[i-1]) / denom

    dp[n-1] = (d[n-1] - a[n-2] * dp[n-2]) / (b[n-1] - a[n-2] * cp[n-2])

    x = np.empty(n, dtype=d.dtype)
    x[-1] = dp[-1]
    for i in range(n-2, -1, -1):
        x[i] = dp[i] - cp[i] * x[i+1]

    return x


def step_numpy_thomas(aA, bA, cA, aB, bB, cB, u, nsteps):
    u = u.copy()
    for _ in range(nsteps):
        rhs = rhs_tridiag(aB, bB, cB, u[1:-1])
        u[1:-1] = thomas_numpy(aA, bA, cA, rhs)
    return u

# Warm-up
_ = step_numpy_thomas(aA, bA, cA, aB, bB, cB, u0, 2)


In [8]:
%%timeit
_ = step_numpy_thomas(aA, bA, cA, aB, bB, cB, u0, nsteps)


11.1 ms ± 559 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
# Numba versions

@njit(cache=True)
def rhs_tridiag_numba(a, b, c, u_int):
    n = u_int.size
    out = b * u_int
    for i in range(1, n):
        out[i] += a[i-1] * u_int[i-1]
    for i in range(0, n-1):
        out[i] += c[i] * u_int[i+1]
    return out


@njit(cache=True)
def thomas_numba(a, b, c, d):
    n = d.size
    cp = np.empty(n-1, dtype=d.dtype)
    dp = np.empty(n, dtype=d.dtype)

    cp[0] = c[0] / b[0]
    dp[0] = d[0] / b[0]

    for i in range(1, n-1):
        denom = b[i] - a[i-1] * cp[i-1]
        cp[i] = c[i] / denom
        dp[i] = (d[i] - a[i-1] * dp[i-1]) / denom

    dp[n-1] = (d[n-1] - a[n-2] * dp[n-2]) / (b[n-1] - a[n-2] * cp[n-2])

    x = np.empty(n, dtype=d.dtype)
    x[-1] = dp[-1]
    for i in range(n-2, -1, -1):
        x[i] = dp[i] - cp[i] * x[i+1]

    return x


@njit(cache=True)
def step_numba_thomas(aA, bA, cA, aB, bB, cB, u, nsteps):
    u = u.copy()
    for _ in range(nsteps):
        rhs = rhs_tridiag_numba(aB, bB, cB, u[1:-1])
        u[1:-1] = thomas_numba(aA, bA, cA, rhs)
    return u

# JIT warm-up (compile)
_ = step_numba_thomas(aA, bA, cA, aB, bB, cB, u0, 2)


In [10]:
%%timeit
_ = step_numba_thomas(aA, bA, cA, aB, bB, cB, u0, nsteps)


160 μs ± 15.4 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [11]:
# Correctness check: all three should match (within floating error)

u_dense   = step_numpy_dense(A, B, u0, nsteps)
u_thomas  = step_numpy_thomas(aA, bA, cA, aB, bB, cB, u0, nsteps)
u_numba   = step_numba_thomas(aA, bA, cA, aB, bB, cB, u0, nsteps)

print('max |dense - thomas|:', np.max(np.abs(u_dense - u_thomas)))
print('max |thomas - numba|:', np.max(np.abs(u_thomas - u_numba)))


max |dense - thomas|: 1.1102230246251565e-15
max |thomas - numba|: 0.0
